# Python - Oracle 연동 - 데이터 조회
## # 01. 준비과정

### #1. 패키지 참조

'cx_oracle', 'sqlalchemy', 'pandas', 'openpyxl', 'xlrd' 패키지가 미리 설치되어있어야한다.

In [16]:
# 'O' 대문자 주의
import cx_Oracle as cx
from sqlalchemy import create_engine
from pandas import DataFrame, read_sql_table

## #02. cx_oracle을 사용한 연동

데이터베이스 연동 과정은 '데이터 베이스 접속 -> SQL 실행 객체(curs) 생성 -> 결과처리 -> 데이터베이스 접속 해제' 의 순서로 진행된다.

### 1. 데이터베이스 접속

In [17]:
dbcon = cx.connect('hr',                 # 사용자 이름
                   'hr',                 # 비밀번호
                   'localhost:1521/xe'   # 데이터베이스 서버 주소
)

### 2. Cursor 객체 생성

In [21]:
cursor = dbcon.cursor()

Help on built-in function fetchall:

fetchall(...) method of cx_Oracle.Cursor instance



### 3. 데이터조회를 위한 SQL문 수행
#### 1) 튜플을 원소로 갖는 리스트 형태로 조회

In [4]:
sql = 'SELECT * FROM department'
try:
    cursor.execute(sql)
    result = cursor.fetchall()
    print(result)   
except Exception as e: # 잘못된 테이블을 입력했을 경우 실패
    print('데이터 조회 실패',e)  

[(210, '응용과학', None), (211, '영문학과', None), (101, '컴퓨터공학과', '1호관'), (102, '멀티미디어학과', '2호관'), (201, '전자공학과', '3호관'), (202, '기계공학과', '4호관')]


#### 2) 딕셔너리를 원소로 갖는 리스트 형태로 조회
cx_oracle은 별도의 딕셔너리 형태를 제공하지 않는다.

cursor객체의 rowfactory 프로퍼티를 직접 재정의 해야 한다.

In [5]:
sql = 'SELECT * FROM department'
try:    
    cursor.execute(sql)

    # 오라클의 경우 튜플을 원소라 갖는 리스트를 생성하는것이 기본값이다.
    # rowfactory 재정의 - 접속 해제 전까지 최초 1회만 수행하면 된다.
    
    cursor.rowfactory = lambda *args: dict(zip([d[0] for d in cursor.description],args)) # 람다식은 오라클 공식 문서에서 배포

    result = cursor.fetchall()
    print(result)
except Exception as e: # 잘못된 테이블을 입력했을 경우 실패
    print('데이터 조회 실패',e)  

[{'DEPTNO': 210, 'DNAME': '응용과학', 'LOC': None}, {'DEPTNO': 211, 'DNAME': '영문학과', 'LOC': None}, {'DEPTNO': 101, 'DNAME': '컴퓨터공학과', 'LOC': '1호관'}, {'DEPTNO': 102, 'DNAME': '멀티미디어학과', 'LOC': '2호관'}, {'DEPTNO': 201, 'DNAME': '전자공학과', 'LOC': '3호관'}, {'DEPTNO': 202, 'DNAME': '기계공학과', 'LOC': '4호관'}]


### 4. 조회결과를 활용한 후속처리

#### 1) 수업에서 배운 내용 활용

In [6]:
with open('department.csv','w',encoding='utf-8') as f:
    for i,v in enumerate(result):
        if i==0:
            keys = list(v.keys())
            titleLine = ','.join(keys)+'\n'
            f.write(titleLine)
        
        values = [str(x) for x in v.values()]
        valueLine = ','.join(values)+'\n'
        f.write(valueLine)

#### 2) 앞으로 배울 내용 맛보기

In [7]:
df = DataFrame(result)
df

,DEPTNO,DNAME,LOC
0,210,응용과학,None
1,211,영문학과,None
2,101,컴퓨터공학과,1호관
3,102,멀티미디어학과,2호관
4,201,전자공학과,3호관
5,202,기계공학과,4호관


In [8]:
df.to_csv('department2.csv', index=False, encoding='utf-8')

In [9]:
df.to_excel('department3.xlsx', index=False)

### 5. 데이터베이스 접속 해제

cursor 객체와 dbcon객체를 닫아서 점유하고 있는 메모리를 컴퓨터에 반납해야한다

자원반납은 생성된 역순으로 진행한다.

In [10]:
try:
    cursor.close()
    dbcon.close()
except Exception as e :
    print('접속 상태가 아닙니다.',e)

## #03. sqlalchemy을 사용한 연동

데이터베이스의 종류를 구분하지 않고 공통된 연동 방법을 제공하는 인터페이스

### 1. 데이터베이스 접속하기
#### 접속 문자열 생성
'oracle+cx_oracle://계정이름:비밀번호@접속주소/SID'

In [11]:
conStr = 'oracle+cx_oracle://hr:hr@localhost:1521/xe'

접속

In [12]:
engine = create_engine(conStr)
dbcon = engine.connect()

### 2. 데이터 조회하기

조회 결과를 pandas의 DataFrame객체 형태로 반환한다.
#### 1) 특정 테이블의 모든 데이터 조회


In [13]:
df = read_sql_table('department', con=dbcon)
df

,deptno,dname,loc
0,210,응용과학,None
1,211,영문학과,None
2,101,컴퓨터공학과,1호관
3,102,멀티미디어학과,2호관
4,201,전자공학과,3호관
5,202,기계공학과,4호관


#### 2) 원하는 컬럼만 조회하기

In [14]:
df = read_sql_table('student',columns=['studno','name','grade','idnum'], con=dbcon)
df

# sql문을 직접쓰지 않는다는 장점 vs 시간이 좀더 걸리고 where절(조건문)을 못넣음

,studno,name,grade,idnum
0,10110,홍길동,1,8501011143098
1,10111,둘리,2,8202021234765
2,10101,전인하,4,7907021369824
3,10102,박미경,1,8405162123648
4,10103,김영균,3,8103211063421
5,10104,지은경,2,8004122298371
6,10105,임유진,2,8301212196482
7,10106,서재진,1,8511291186273
8,10107,이광훈,4,8109131276431
9,10108,류민정,2,8108192157498


### 3. 데이터베이스 접속 해제


In [15]:
dbcon.close()